In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

X_PATH = "processed/preprocessed_X_seq.npy"
Y_PATH = "processed/preprocessed_y_seq.npy"

BATCH_SIZE = 32
EPOCHS = 15
LR = 1e-3

os.makedirs("results", exist_ok=True)


Using device: cuda


In [2]:
X_seq = np.load(X_PATH)
y_seq = np.load(Y_PATH)

print("X:", X_seq.shape)
print("y:", y_seq.shape)

def create_splits(X, y, train_ratio=0.7, val_ratio=0.15):
    N = X.shape[0]
    n_train = int(N * train_ratio)
    n_val   = int(N * val_ratio)
    return (
        (X[:n_train], y[:n_train]),
        (X[n_train:n_train+n_val], y[n_train:n_train+n_val]),
        (X[n_train+n_val:], y[n_train+n_val:])
    )

(X_train, y_train), (X_val, y_val), (X_test, y_test) = create_splits(X_seq, y_seq)
print(len(X_train), len(X_val), len(X_test))


X: (78773, 32, 52)
y: (78773, 32)
55141 11815 11817


In [3]:
class FlowDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return torch.tensor(self.X[idx], dtype=torch.float32), torch.tensor(self.y[idx], dtype=torch.long)

train_loader = DataLoader(FlowDataset(X_train, y_train), batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(FlowDataset(X_val, y_val),     batch_size=BATCH_SIZE, shuffle=False)
test_loader  = DataLoader(FlowDataset(X_test, y_test),   batch_size=BATCH_SIZE, shuffle=False)


In [4]:
from torch_geometric.nn import SAGEConv

class GNN_IDS(nn.Module):
    def __init__(self, in_dim, emb_dim=128, hidden_dim=128, layers=2, num_classes=2):
        super().__init__()
        self.embed = nn.Sequential(
            nn.Linear(in_dim, 128),
            nn.ReLU(),
            nn.Linear(128, emb_dim)
        )
        convs = []
        for i in range(layers):
            convs.append(SAGEConv(emb_dim if i == 0 else hidden_dim,
                                  emb_dim if i == layers-1 else hidden_dim))
        self.convs = nn.ModuleList(convs)
        self.relu = nn.ReLU()
        self.classifier = nn.Linear(emb_dim, num_classes)

    def build_edges(self, B, L, device):
        nodes = torch.arange(L, device=device)
        src, dst = torch.meshgrid(nodes, nodes, indexing="ij")
        mask = src != dst
        src, dst = src[mask], dst[mask]
        edges = []
        for b in range(B):
            o = b * L
            edges.append(torch.stack([src + o, dst + o]))
        return torch.cat(edges, dim=1)

    def forward(self, x):
        B, L, _ = x.shape
        x = self.embed(x)
        edge_index = self.build_edges(B, L, x.device)
        h = x.reshape(B * L, -1)
        for conv in self.convs:
            h = self.relu(conv(h, edge_index))
        h = h.reshape(B, L, -1)
        return self.classifier(h)


c:\Users\Admin\anaconda3\envs\ids\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
flat = y_train.reshape(-1)
counts = np.bincount(flat, minlength=2).astype(float)
weights = counts.sum() / (2 * counts + 1e-8)
class_weights = torch.tensor(weights, device=DEVICE, dtype=torch.float32)
print("Class weights:", class_weights)


Class weights: tensor([0.5757, 3.8037], device='cuda:0')


In [6]:
model = GNN_IDS(in_dim=X_seq.shape[-1]).to(DEVICE)
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
best_val_acc = -1
best_model = "results/gnn_best.pth"

def acc_fn(logits, y):
    preds = logits.argmax(dim=-1)
    return (preds == y).float().mean().item()

for epoch in range(1, EPOCHS + 1):
    model.train()
    tl, ta = 0, 0
    for xb, yb in train_loader:
        xb, yb = xb.to(DEVICE), yb.to(DEVICE)
        logits = model(xb)
        loss = criterion(logits.view(-1, 2), yb.view(-1))
        optimizer.zero_grad(); loss.backward(); optimizer.step()
        tl += loss.item()
        ta += acc_fn(logits, yb)
    tl /= len(train_loader); ta /= len(train_loader)

    model.eval()
    vl, va = 0, 0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            logits = model(xb)
            loss = criterion(logits.view(-1, 2), yb.view(-1))
            vl += loss.item()
            va += acc_fn(logits, yb)
    vl /= len(val_loader); va /= len(val_loader)

    if va > best_val_acc:
        best_val_acc = va
        torch.save(model.state_dict(), best_model)
        print(f"💾 Saved best model at epoch {epoch} | Val Acc={va:.4f}")

    print(f"Epoch {epoch}/{EPOCHS} | Train Loss={tl:.4f} Acc={ta:.4f} | Val Loss={vl:.4f} Acc={va:.4f}")

print("Training DONE.")


💾 Saved best model at epoch 1 | Val Acc=0.8690
Epoch 1/15 | Train Loss=0.0418 Acc=0.9894 | Val Loss=1.5623 Acc=0.8690
Epoch 2/15 | Train Loss=0.0214 Acc=0.9942 | Val Loss=3.0499 Acc=0.6891
Epoch 3/15 | Train Loss=0.0173 Acc=0.9950 | Val Loss=1.8999 Acc=0.8078
Epoch 4/15 | Train Loss=0.0178 Acc=0.9951 | Val Loss=1.8214 Acc=0.8229
💾 Saved best model at epoch 5 | Val Acc=0.8768
Epoch 5/15 | Train Loss=0.0152 Acc=0.9957 | Val Loss=1.5741 Acc=0.8768
Epoch 6/15 | Train Loss=0.0153 Acc=0.9957 | Val Loss=1.3713 Acc=0.8668
Epoch 7/15 | Train Loss=0.0134 Acc=0.9960 | Val Loss=1.5522 Acc=0.8745
Epoch 8/15 | Train Loss=0.0132 Acc=0.9960 | Val Loss=1.6354 Acc=0.8701
Epoch 9/15 | Train Loss=0.0146 Acc=0.9958 | Val Loss=1.8136 Acc=0.8447
Epoch 10/15 | Train Loss=0.0135 Acc=0.9962 | Val Loss=1.1742 Acc=0.8637
💾 Saved best model at epoch 11 | Val Acc=0.8776
Epoch 11/15 | Train Loss=0.0122 Acc=0.9964 | Val Loss=2.0053 Acc=0.8776
Epoch 12/15 | Train Loss=0.0116 Acc=0.9964 | Val Loss=1.6519 Acc=0.8758
💾 S

In [7]:
model.load_state_dict(torch.load(best_model))
model.eval()

y_true, y_pred = [], []
with torch.no_grad():
    for xb, yb in test_loader:
        xb = xb.to(DEVICE)
        logits = model(xb)
        preds = logits.argmax(dim=-1)
        y_true.extend(yb.reshape(-1).tolist())
        y_pred.extend(preds.cpu().reshape(-1).tolist())

acc  = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred, zero_division=0)
rec  = recall_score(y_true, y_pred, zero_division=0)
f1   = f1_score(y_true, y_pred, zero_division=0)

print("🔍 GNN Test Metrics:")
print("Accuracy :", acc)
print("Precision:", prec)
print("Recall   :", rec)
print("F1 score :", f1)

import pandas as pd
df = pd.DataFrame([[ "GNN", acc, prec, rec, f1 ]],
                  columns=["Model", "Accuracy", "Precision", "Recall", "F1"])
df.to_csv("results/GNN.csv", index=False)
print("📁 Exported to results/GNN.csv")
df


🔍 GNN Test Metrics:
Accuracy : 0.9546945079123297
Precision: 0.878679190297922
Recall   : 0.7698214741638353
F1 score : 0.8206561564390846
📁 Exported to results/GNN.csv


,Model,Accuracy,Precision,Recall,F1
0,GNN,0.954695,0.878679,0.769821,0.820656
